In [45]:
#Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc, roc_auc_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
#Load dataset
import pandas as pd
df1 = pd.read_csv("cleanprojectdataset.csv")

In [47]:
print(df1)

                                                  Tweet    Text Label
0     .omg why are poc wearing fugly blue contacts s...  Non-Bullying
1     .Sorry but most of the runners popular right n...  Non-Bullying
2     .those jeans are hideous, and I?m afraid he?s ...  Non-Bullying
3     .I had to dress up for a presentation in class...  Non-Bullying
4     .Am I the only one who thinks justin bieber is...  Non-Bullying
...                                                 ...           ...
1060  No we are not, But you are a race baiting libt...      Bullying
1061  you wont get anyone for this challenge., after...      Bullying
1062  I will follow you if you are not a libtard,Mus...      Bullying
1063  michaelianblack Ur a child, an ostrich w/ your...      Bullying
1064  FoxNews. not to all the ppl I know that live t...      Bullying

[1065 rows x 2 columns]


In [32]:
#Tokenize words and labels into lists

Tweet = []
Labels = []

for row in df1['Tweet']:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ["''",'``',"rt","https","’","“","”","\u200b","--","n't","'s","...","//t.c" ]
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Tweet.append(lemma_list)

    for row in df1["Text Label"]:
        Labels.append(row)

In [33]:
#combine them to create bag of words
combined = zip(Tweet, Labels)

In [34]:
#Create bag of words and dictionary object
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [35]:
#Key, Value Pair into new list for modeling
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))

In [36]:
+#random shuffle
import random
random.shuffle(Final_Data)
print(len(Final_Data))

1065


In [28]:
train_set, test_set = Final_Data[0:747], Final_Data[747:]

import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure)
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)
nb_classifier.show_most_informative_features(10)

from nltk.classify.util import accuracy
print(accuracy(nb_classifier, test_set))

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
    
for i, (Final_Data, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = nb_classifier.classify(Final_Data)
    testsets[observed].add(i)
    
print('bullying precision:', precision(refsets['Bullying'], testsets['Bullying']))
print('bullying recall:', recall(refsets['Bullying'], testsets['Bullying']))
print('bullying F-measure:', f_measure(refsets['Bullying'], testsets['Bullying']))
print('not-bullying precision:', precision(refsets['Non-Bullying'], testsets['Non-Bullying']))
print('not-bullying recall:', recall(refsets['Non-Bullying'], testsets['Non-Bullying']))
print('not-bullying F-measure:', f_measure(refsets['Non-Bullying'], testsets['Non-Bullying']))

Most Informative Features
                     low = True           Bullyi : Non-Bu =      9.2 : 1.0
               worthless = True           Bullyi : Non-Bu =      7.9 : 1.0
                 someone = True           Non-Bu : Bullyi =      7.9 : 1.0
                      iq = True           Bullyi : Non-Bu =      6.6 : 1.0
                   lying = True           Bullyi : Non-Bu =      5.4 : 1.0
                      ur = True           Bullyi : Non-Bu =      5.4 : 1.0
                 libtard = True           Bullyi : Non-Bu =      5.3 : 1.0
                    cunt = True           Bullyi : Non-Bu =      4.8 : 1.0
                  person = True           Non-Bu : Bullyi =      4.8 : 1.0
                   piece = True           Bullyi : Non-Bu =      4.7 : 1.0
0.6477987421383647
bullying precision: 0.5403726708074534
bullying recall: 0.696
bullying F-measure: 0.6083916083916083
not-bullying precision: 0.7579617834394905
not-bullying recall: 0.616580310880829
not-bullying F-measure